In [19]:
# JUPYTER CELL 1
import importlib, sys
sys.path.append(".")

# If you tweak data.py while iterating, uncomment the next two lines:
# %load_ext autoreload
# %autoreload 2

import Data  # executes your Data.py once (downloads prices, builds DataFrames)

# Pull what we need
close_df = Data.close_df        # wide Close matrix with (Sector, Ticker) MultiIndex columns
long_df  = Data.long_df         # long-format table
sectors  = Data.sectors         # dict: sector -> tickers

# Quick sanity check
close_df.head()


Sector            Tech                         Finance                         \
Ticker            AAPL        MSFT    NVDA         JPM        BAC          MA   
Date                                                                            
2022-09-19  154.479996  244.520004  13.382  118.160004  34.689999  314.029999   
2022-09-20  156.899994  242.449997  13.176  115.830002  34.169998  313.269989   
2022-09-21  153.720001  238.949997  13.261  112.489998  33.150002  304.500000   
2022-09-22  152.740005  240.979996  12.561  111.209999  32.500000  298.359985   
2022-09-23  150.429993  237.919998  12.516  109.139999  31.730000  293.579987   

Sector      Healthcare                        Consumer Goods              \
Ticker             JNJ        PFE        ABBV            PEP         MCD   
Date                                                                       
2022-09-19  166.279999  45.439999  142.660004     168.729996  257.010010   
2022-09-20  164.970001  44.770000  141.770004     168.919998  255.399994   
2022-09-21  163.279999  43.919998  140.309998     168.440002  251.080002   
2022-09-22  166.179993  44.570000  143.009995     168.600006  247.940002   
2022-09-23  166.720001  44.080002  143.059998     168.520004  245.949997   

Sector                     Energy                         
Ticker             NKE        XOM       SHEL         CVX  
Date                                                      
2022-09-19  107.209999  93.199997  52.490002  156.899994  
2022-09-20  102.419998  92.440002  52.849998  156.279999  
2022-09-21   99.790001  90.949997  52.330002  155.009995  
2022-09-22   98.550003  90.570000  52.340000  154.889999  
2022-09-23   97.019997  85.750000  48.110001  144.770004

In [20]:
# JUPYTER CELL 2
import pandas as pd
import numpy as np

def get_close_series(close_df: pd.DataFrame, ticker: str) -> pd.Series:
    """
    Extract a single ticker's Close price Series from a (Sector, Ticker) MultiIndex wide DataFrame.
    """
    sub = close_df.xs(ticker, level="Ticker", axis=1)
    s = sub.iloc[:, 0] if isinstance(sub, pd.DataFrame) else sub
    s.name = ticker
    return s.sort_index()

def sma(series: pd.Series, window: int, min_periods: int | None = None) -> pd.Series:
    """
    Simple Moving Average for a 1-D Series.
    """
    return series.rolling(window=window, min_periods=min_periods or window).mean()

def sma_wide(close_df: pd.DataFrame, window: int, min_periods: int | None = None) -> pd.DataFrame:
    """
    SMA for every (Sector, Ticker) column in the wide Close matrix.
    Returns a DataFrame with the same MultiIndex columns.
    """
    out = close_df.sort_index().rolling(window=window, min_periods=min_periods or window).mean()
    out.columns = close_df.columns.set_names(["Sector", "Ticker"])
    return out

def daily_returns(series: pd.Series) -> pd.Series:
    """
    Simple daily returns r_t = (P_t - P_{t-1}) / P_{t-1} = pct_change().
    """
    return series.pct_change()

def daily_returns_wide(close_df: pd.DataFrame) -> pd.DataFrame:
    """
    Daily returns for the entire wide Close matrix.
    """
    out = close_df.pct_change()
    out.columns = close_df.columns.set_names(["Sector", "Ticker"])
    return out


In [21]:
# JUPYTER CELL 3
from dataclasses import dataclass

@dataclass
class RunStats:
    up_runs_count: int
    down_runs_count: int
    up_total_days: int
    down_total_days: int
    longest_up_run: int
    longest_down_run: int

def _runs_from_signs(ret: pd.Series) -> RunStats:
    """
    Given a return series (with NaNs allowed), compute up/down run stats.
    - Up day: ret > 0
    - Down day: ret < 0
    Flat (==0) breaks runs but isn't counted as up or down.
    """
    s = ret.copy().fillna(0.0)

    up_runs_count = down_runs_count = 0
    up_total = down_total = 0
    longest_up = longest_down = 0

    curr_sign = 0   # 1 for up-run, -1 for down-run, 0 for none
    curr_len = 0

    for x in s:
        sign = 1 if x > 0 else (-1 if x < 0 else 0)

        if sign == 0:
            # flat day → close out any ongoing run
            if curr_sign == 1:
                up_runs_count += 1
                up_total += curr_len
                longest_up = max(longest_up, curr_len)
            elif curr_sign == -1:
                down_runs_count += 1
                down_total += curr_len
                longest_down = max(longest_down, curr_len)
            curr_sign = 0
            curr_len = 0
            continue

        # if continuing the same run
        if sign == curr_sign:
            curr_len += 1
        else:
            # close previous run
            if curr_sign == 1:
                up_runs_count += 1
                up_total += curr_len
                longest_up = max(longest_up, curr_len)
            elif curr_sign == -1:
                down_runs_count += 1
                down_total += curr_len
                longest_down = max(longest_down, curr_len)
            # start new run
            curr_sign = sign
            curr_len = 1

        # special case: starting fresh
        if curr_sign == 0:
            curr_sign = sign
            curr_len = 1

    # close trailing run
    if curr_len > 0:
        if curr_sign == 1:
            up_runs_count += 1
            up_total += curr_len
            longest_up = max(longest_up, curr_len)
        elif curr_sign == -1:
            down_runs_count += 1
            down_total += curr_len
            longest_down = max(longest_down, curr_len)

    return RunStats(
        up_runs_count=up_runs_count,
        down_runs_count=down_runs_count,
        up_total_days=up_total,
        down_total_days=down_total,
        longest_up_run=longest_up,
        longest_down_run=longest_down,
    )

def runs_for_ticker(close_df: pd.DataFrame, ticker: str) -> RunStats:
    """
    Compute run stats for a single ticker based on close-to-close changes.
    """
    s_close = get_close_series(close_df, ticker)
    ret = daily_returns(s_close)
    return _runs_from_signs(ret)

def runs_summary_all(close_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute run stats for every ticker. Returns a tidy DataFrame indexed by ticker
    (with a 'Sector' column for convenience).
    """
    tickers = close_df.columns.get_level_values("Ticker").unique().tolist()
    sectors = close_df.columns.get_level_values("Sector").unique().tolist()

    records = []
    for t in tickers:
        stats = runs_for_ticker(close_df, t)
        sec = close_df.xs(t, level="Ticker", axis=1).columns[0]  # sector label
        records.append({
            "Ticker": t,
            "Sector": sec,
            "UpRuns": stats.up_runs_count,
            "DownRuns": stats.down_runs_count,
            "UpDaysTotal": stats.up_total_days,
            "DownDaysTotal": stats.down_total_days,
            "LongestUp": stats.longest_up_run,
            "LongestDown": stats.longest_down_run,
        })
    return pd.DataFrame.from_records(records).set_index("Ticker").sort_index()


In [22]:
# JUPYTER CELL 4
def max_profit_buy_sell_ii(series: pd.Series) -> float:
    """
    LeetCode 122 — Best Time to Buy and Sell Stock II.
    Max profit with unlimited transactions (no overlapping).
    Equivalent to summing all positive price differences.
    """
    s = series.dropna()
    deltas = s.diff().fillna(0)
    return float(deltas[deltas > 0].sum())

def max_profit_all(close_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute max profit II for every ticker. Returns a DataFrame with 'Profit' and 'Sector'.
    """
    rows = []
    for t in close_df.columns.get_level_values("Ticker").unique():
        s_close = get_close_series(close_df, t)
        profit = max_profit_buy_sell_ii(s_close)
        sector = close_df.xs(t, level="Ticker", axis=1).columns[0]
        rows.append({"Ticker": t, "Sector": sector, "Profit": profit})
    return pd.DataFrame(rows).set_index("Ticker").sort_values("Profit", ascending=False)


In [23]:
# JUPYTER CELL 5
ticker = "AAPL"
window = 5

# 1) SMA
aapl_close = get_close_series(close_df, ticker)
aapl_sma5  = sma(aapl_close, window=window)

# 2) Up/Down runs
aapl_runs = runs_for_ticker(close_df, ticker)

# 3) Daily returns
aapl_returns = daily_returns(aapl_close)

# 4) Max profit II
aapl_profit = max_profit_buy_sell_ii(aapl_close)

print(f"[{ticker}] SMA({window}) last value:", float(aapl_sma5.dropna().iloc[-1]))
print(f"[{ticker}] Runs → {aapl_runs}")
print(f"[{ticker}] Last return:", float(aapl_returns.dropna().iloc[-1]))
print(f"[{ticker}] Max Profit (Buy/Sell II): {aapl_profit:,.2f}")


[AAPL] SMA(5) last value: 235.58800048828124
[AAPL] Runs → RunStats(up_runs_count=182, down_runs_count=181, up_total_days=404, down_total_days=345, longest_up_run=8, longest_down_run=8)
[AAPL] Last return: 0.0035272375318422533
[AAPL] Max Profit (Buy/Sell II): 893.24
